# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. Zhang  ->  R. Zhang  |  ['R. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 40 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.12386


extracting tarball to tmp_2407.12386... done.


R. Zhang  ->  R. Zhang  |  ['R. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Found 99 bibliographic references in tmp_2407.12386/The_Extinction_curve_revealed_by_Gaia_XP_spectra.bbl.
syntax error in line 326: '=' expected
Retrieving document from  https://arxiv.org/e-print/2407.12585


extracting tarball to tmp_2407.12585...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2082/1211882699.py:51: LatexWarning: 2407.12585 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.12386-b31b1b.svg)](https://arxiv.org/abs/2407.12386) | **An Empirical Extinction Curve Revealed by Gaia XP Spectra and LAMOST**  |
|| <mark>R. Zhang</mark>, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-07-18*|
|*Comments*| *11 pages, 5 figures, accepted for publication in ApJ*|
|**Abstract**|            We present a direct measurement of extinction curves using corrected $Gaia$ XP spectra of the common sources in $Gaia$ DR3 and LAMOST DR7. Our analysis of approximately 370 thousand high-quality samples yielded a high-precision average extinction curve for the Milky Way. After incorporating infrared photometric data from 2MASS and WISE, the extinction curve spans wavelengths from 0.336 to 4.6 $\mu$m. We determine an average $R_{55}$ of $2.730 \pm 0.007$, corresponding to $R_V= 3.073 \pm 0.009$, and a near-infrared power-law index $\alpha$ of $1.935 \pm 0.037$. Our study confirmed some intermediate-scale structures within the optical range. Two new features were identified at 540 and 769 nm, and their intensities exhibited a correlation with extinction and $R_V$. This extinction curve can be used to investigate the characteristics of dust and enhance the extinction correction of Milky Way stars. A Python package for this extinction curve is available.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.12585-b31b1b.svg)](https://arxiv.org/abs/2407.12585) | **A global view on star formation: The GLOSTAR Galactic plane survey. XI. Radio source catalog IV: $2^\circ < \ell < 28^\circ$, $36^\circ < \ell < 60^\circ$ and $|b| < 1^\circ$**  |
|| S.-N. X. Medina, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2024-07-18*|
|*Comments*| *21 pages, 18 figures, 7 tables, accepted to be published in the Astronomy & Astrophysics journal*|
|**Abstract**|            The GLOSTAR survey studies star formation with the VLA and the Effelsberg 100m telescope in the Galactic plane (-2d<l<60d; |b|<1d) and the Cygnus X region with unprecedented sensitivity in both flux density (~50uJy/beam) and the capability of detecting emission with angular scales in the range from 1" to the largest radio structures in the Galaxy. We provide a complete GLOSTAR-VLA D-configuration radio source catalog for the covered part of the Galactic disk. A catalog for the pilot region (28d<l<36d) has been published in a previous paper and here we present the complementary catalog for the area within 2d<l<28d, 36d<l<60d and |b|<1d. Observations were taken with the VLA in a 4-8GHz band to image 100 degrees$^2$ of the inner Galactic disk at a reference frequency of 5.8GHz, using 260h of telescope time. We determined spectral indices inside the observed band and in the frequency range 1.4-5.8GHz by complementing our results with those from the THOR survey (1-2GHz). The final images have an angular resolution of 18" and an average sensitivity of 123uJy/beam. The sensitivity is better (~60uJy/beam) in areas free of extended emission. The Galactic disk catalog presented in this work, consists of 11211 radio sources. Of these, 1965 are known large-scale structure sources such as star-forming region complexes, well-known SNRs, SNR candidates or parts thereof. The remaining 9227 are discrete individual sources. Source parameters, namely flux densities, sizes, spectral indices, and classifications are reported. We identify 769 HII region candidates, 359 are newly classified as such. The mean value of spectral indices of 225 HII regions is 0.14$\pm$0.02, consistent with most of them emitting optically thin thermal radio emission. Combining our results with the previously published catalog of the pilot region, the final GLOSTAR-VLA D-configuration catalog contains 12981 radio sources.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.12386/./5.1_Median_Extinction_Curve.png', 'tmp_2407.12386/./5.2_Parameter_relevance.png', 'tmp_2407.12386/./7.3_CERs,_extinction_curve,_and_models.png']
copying  tmp_2407.12386/./5.1_Median_Extinction_Curve.png to _build/html/
copying  tmp_2407.12386/./5.2_Parameter_relevance.png to _build/html/
copying  tmp_2407.12386/./7.3_CERs,_extinction_curve,_and_models.png to _build/html/
exported in  _build/html/2407.12386.md
    + _build/html/tmp_2407.12386/./5.1_Median_Extinction_Curve.png
    + _build/html/tmp_2407.12386/./5.2_Parameter_relevance.png
    + _build/html/tmp_2407.12386/./7.3_CERs,_extinction_curve,_and_models.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# An Empirical Extinction Curve Revealed by Gaia XP Spectra and LAMOST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.12386-b31b1b.svg)](https://arxiv.org/abs/2407.12386)<mark>Appeared on: 2024-07-18</mark> -  _11 pages, 5 figures, accepted for publication in ApJ_

</div>
<div id="authors">

<mark>R. Zhang</mark>, et al. -- incl., <mark>X. Zhang</mark>

</div>
<div id="abstract">

**Abstract:** We present a direct measurement of extinction curves using corrected $\gaia$ XP spectra \citep{2024ApJS..271...13H} of the common sources in $\gaia$ DR3 and LAMOST DR7.Our analysis of approximately 370 thousand high-quality samples yielded a high-precision average extinction curve for the Milky Way.After incorporating infrared photometric data from 2MASS and WISE, the extinction curve spans wavelengths from 0.336 to 4.6 $\mu$ m.We determine an average $R_{55}$ of $2.730 \pm 0.007$ , corresponding to $\RV = 3.073 \pm 0.009$ , and a near-infrared power-law index $\alpha$ of $1.935 \pm 0.037$ .Our study confirmed some intermediate-scale structures within the optical range.Two new features were identified at 540 and 769 nm, and their intensities exhibited a correlation with extinction and $\RV$ .This extinction curve can be used to investigate the characteristics of dust and enhance the extinction correction of Milky Way stars.A Python package for this extinction curve is available \citep{ruoyi_zhang_2024_12621834} .

</div>

<div id="div_fig1">

<img src="tmp_2407.12386/./5.1_Median_Extinction_Curve.png" alt="Fig2" width="100%"/>

**Figure 2. -** The median extinction curve and its comparison with literature.
    The black line is the median extinction curve, while the gray shadow indicates the distribution range for 25\% to 75\% of stars.
    The models from ZG24, \citetalias{2019ApJ...886..108F}, \citetalias{2023ApJ...950...86G}, and \citetalias{1989ApJ...345..245C} are illustrated by red, pink, light blue, and blue lines, respectively.
    The gray dashed lines and the wavelength labels indicate the locations of the possible ISSs. (*fig-MEC*)

</div>
<div id="div_fig2">

<img src="tmp_2407.12386/./5.2_Parameter_relevance.png" alt="Fig3" width="100%"/>

**Figure 3. -** The difference between the average extinction curve of samples within varying parameter ranges and the overall median curve.
    From top to bottom, these panels display the impact of the following parameters: G band magnitude, $\teff$, $\logg$, $\ebv$, and $\RV$.
    Colors represent different ranges of each parameter.
     (*fig-relevance*)

</div>
<div id="div_fig3">

<img src="tmp_2407.12386/./7.3_CERs,_extinction_curve,_and_models.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
    Comparisons between the median extinction curve and literature models along with photometric data.
    _ Top panel_: The black solid and dashed lines are the median extinction and infrared extension curves, respectively. The red, pink, light blue, and blue lines are extinction models from ZG24, \citetalias{2019ApJ...886..108F}, \citetalias{2023ApJ...950...86G}, and \citetalias{1989ApJ...345..245C}, respectively. Blue open circles indicate the median $k(\lambda-55)$ and median $\lambda_{\rm{eff}}$ for each passband, with error bars included, although they are nearly imperceptible. The names of the corresponding passbands are labeled at the top. The gray dashed lines and the wavelength labels indicate the locations of the possible ISSs.
    _ Middle panel_: The residuals obtained by subtracting the median extinction curve from the four models in the top panel.
    _ Bottom panel_: The residual curves corresponding to the best-fit $\RV$ values.
     (*fig-liter*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.12386"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

138  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
